In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set_style("darkgrid")
matplotlib.rc('font', size=20)
matplotlib.rc('axes', titlesize=20)
matplotlib.rc('axes', labelsize=20)
matplotlib.rc('xtick', labelsize=20)
matplotlib.rc('ytick', labelsize=20)
matplotlib.rc('legend', fontsize=20)
matplotlib.rc('figure', titlesize=20)
%matplotlib inline

In [4]:
train_df = pd.read_csv("/Users/sominwadhwa/Work/Kaggle_Data/Russian Housing Market/train.csv"
                      , parse_dates = ['timestamp'])
test_df = pd.read_csv("/Users/sominwadhwa/Work/Kaggle_Data/Russian Housing Market/test.csv",
                     parse_dates = ['timestamp'])
macro_df = pd.read_csv("/Users/sominwadhwa/Work/Kaggle_Data/Russian Housing Market/macro.csv",
                      parse_dates = ['timestamp'])

In [5]:
train_df = pd.merge(train_df,macro_df,on='timestamp', how='left')
test_df = pd.merge(test_df,macro_df,on='timestamp',how='left')
print (train_df.shape,test_df.shape)

(30471, 391) (7662, 390)


In [6]:
print (train_df.info(),test_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30471 entries, 0 to 30470
Columns: 391 entries, id to apartment_fund_sqm
dtypes: datetime64[ns](1), float64(213), int64(159), object(18)
memory usage: 91.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7662 entries, 0 to 7661
Columns: 390 entries, id to apartment_fund_sqm
dtypes: datetime64[ns](1), float64(210), int64(161), object(18)
memory usage: 22.9+ MB
None None
